In [31]:
import sys
sys.path.append('..')
from lre.data import Relation, RelationSample
from lre.operators import JacobianIclEstimator, Word2VecIclEstimator
import lre.functional as functional

In [32]:
import lre.models as models
device = "mps"
mt = models.load_model("gptj", device=device, fp16=True)

In [26]:
#We want to evaluate each subject with any of the available objects.
animal_path = 'data/enckno/E06 [animal - young].txt'
subjects = []
subject_object_pairs = []
all_pairs = []

def relation_from_path(path, relation_name, prompt):
    with open(path, "r") as f:
        lines = f.readlines()
        lines = [line.replace('\n','') for line in lines]

        for line in lines:
            a, bs = line.split('\t')
            subjects.append(a)
            bs = bs.split('/')
            subject_object_pairs.append((a,bs[0]))
            for b in bs:
                all_pairs.append((a,b))

    pairs = subject_object_pairs
    RelationSamples = [RelationSample(*pair) for pair in pairs]
    rel = Relation(
                    name=relation_name,
                    prompt_templates=[prompt],
                    prompt_templates_zs=[],
                    samples=
                    RelationSamples
                )
    return rel

animal_rel = relation_from_path(animal_path, "animal-youth", "The young version of {} is")

In [27]:
def test_operator_on_relation(operator, relation, mt, h_layer, z_layer):
    k = 5
    #construct the operator
    operator = operator(mt=mt, h_layer=h_layer, z_layer=z_layer)
    
    #uses the first prompt template
    prompt_template = relation.prompt_templates[0]

    #assemble in-context prompts
    clozed_prompts = []
    for x in relation.samples:
        clozed_samples = [s for s in relation.samples if s != x]
        cloze_template = functional.make_prompt(
            prompt_template=prompt_template,
            subject="{}",
            examples = clozed_samples
            )
        cloze_prompt = cloze_template.format(x.subject)
        clozed_prompts.append(cloze_prompt)

    for prompt in (clozed_prompts):
        print(f'Prompt: \n{prompt}\n')

    outputs_lm = functional.predict_next_token(mt=mt, prompt=clozed_prompts, k=k)
    preds_lm =  [[x.token for x in xs] for xs in outputs_lm]
    print(preds_lm)


In [28]:
#Word2Vec returns indistinguishable results for these three pairs.

#test_operator_on_relation(JacobianIclEstimator, rel, mt, 5, 27)
test_operator_on_relation(Word2VecIclEstimator, animal_rel, mt, 5, 27)

Prompt: 
The young version of badger is kit
The young version of bear is cub
The young version of beaver is kit
The young version of bee is larva
The young version of beetle is larva
The young version of buffalo is calf
The young version of butterfly is larva
The young version of camel is calf
The young version of cat is kitten
The young version of cattle is calf
The young version of chimpanzee is baby
The young version of cicada is nymph
The young version of cockroach is nymph
The young version of cricket is larva
The young version of deer is fawn
The young version of dog is puppy
The young version of duck is duckling
The young version of elephant is calf
The young version of ferret is kit
The young version of fish is fingerling
The young version of fly is grub
The young version of fox is cub
The young version of goat is kid
The young version of goldfish is fingerling
The young version of gorilla is infant
The young version of herring is fingerling
The young version of horse is foal
T

RuntimeError: MPS backend out of memory (MPS allocated: 16.25 GB, other allocations: 1.82 GB, max allowed: 18.13 GB). Tried to allocate 651.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).